делает датафрейм

In [1]:
import pandas as pd
import numpy as np

microtasks_df = pd.read_csv('D:\\Desktop\\test_ya\\data_task4_old.txt', sep = '\t')
print (microtasks_df.head())

    login          tid  Microtasks          assigned_ts            closed_ts
0  login0  190563850.0         4.0  2017-04-20 12:09:39  2017-04-20 13:13:01
1  login0  190561754.0         1.0  2017-04-20 12:10:30  2017-04-20 12:28:29
2  login0  190565906.0         4.0  2017-04-20 12:21:31  2017-04-20 13:30:10
3  login0  190560246.0         1.0  2017-04-20 12:28:30  2017-04-20 13:13:56
4  login0  190562168.0         2.0  2017-04-20 12:28:42  2017-04-20 13:14:50


проверяет типы столбцов

In [2]:
microtasks_df.dtypes

login           object
tid            float64
Microtasks     float64
assigned_ts     object
closed_ts       object
dtype: object

Меняет типы двух последний столбов на временные

In [5]:
microtasks_df['assigned_ts'] = pd.to_datetime(microtasks_df.assigned_ts)
microtasks_df['closed_ts'] = pd.to_datetime(microtasks_df.closed_ts) 

Через разницу столбцов, считает время, потраченное на каждое задание, выводит его в новый столбец 'time spent'

In [6]:
microtasks_df['time_spent'] = microtasks_df['closed_ts'] - microtasks_df['assigned_ts']

In [7]:
print (microtasks_df.head())

    login          tid  Microtasks         assigned_ts           closed_ts  \
0  login0  190563850.0         4.0 2017-04-20 12:09:39 2017-04-20 13:13:01   
1  login0  190561754.0         1.0 2017-04-20 12:10:30 2017-04-20 12:28:29   
2  login0  190565906.0         4.0 2017-04-20 12:21:31 2017-04-20 13:30:10   
3  login0  190560246.0         1.0 2017-04-20 12:28:30 2017-04-20 13:13:56   
4  login0  190562168.0         2.0 2017-04-20 12:28:42 2017-04-20 13:14:50   

  time_spent  
0   01:03:22  
1   00:17:59  
2   01:08:39  
3   00:45:26  
4   00:46:08  


'time_spent' становится средним временем на выполнение одного микрозадания задания (по строке)

In [10]:
microtasks_df['time_spent'] = microtasks_df['time_spent'] / np.timedelta64(1, 's') / microtasks_df['Microtasks']

Считает среднее время на выполнение одного задания (по всей таблице)

In [11]:
time_for_microtask = microtasks_df['time_spent'].sum () / len(microtasks_df)

In [12]:
print (time_for_microtask)

568.1515392052839


Оплата по 30 секунд - значит, делим на 30

In [13]:
print (time_for_microtask / 30)

18.93838464017613


Таким образом, мы вычислили среднее время выполнения одного микрозадания ~568 с, и, соотвественно, оплату за него ~18,9N.

Поскольку мы знаем, что во время резерва задания асессор может заниматься чем-то другим, но невозможна ситуация, где он в это время работает сверх (если только он не использует машину времени, чтобы в тот же промежуток времени сделать в два раза больше), то нужно решить необходимость уменьшить 18,9N. Если, например, N предназначается для 30 секунд только работы и само по себе не учитывает перерывы - должны ли мы их учитывать в итоговой оплате за микрозадание?
Если да, надо будет смотреть в зависимости от того, что мы будем считать справедливым возможным перерывом в это время резерва:
1. T секунд в один промежуток времени (например, 5 минут каждый час работы не должны оплачиваться) - в этом случае можно рассчитать общее время работы, вычесть эти "5 минут из каждого часа", а уже потом вычислять среднее за микрозадание.
2. считать по асессорам (например, 5% от всего времени каждого асессора считается перерывом) - тогда нужно сгруппировывать данные по времени каждого асессора, вычесть оттуда 5%, а уже потом считать среднее время на микрозадание. 
3. вычитать "перерывное время" непосредственно из среднего времени по микротаску (допустим, 60с из 568с - перерыв, тогда оплата будет 16,9N)